In [1]:
#Import Library
import os
import shutil
import glob
import cv2

In [2]:
#Drive Mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#GPU 
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [4]:
#데이터를 배열로 초기화
data = {}
data['crack'] = []
data['uncrack'] = []

In [5]:
#Crack Data
src = '/content/drive/MyDrive/딥러닝 오소리 /Data Set/10000 DataSet/Positive'
for jpgImage in glob.iglob(os.path.join(src, '*jpg')):
    data['crack'].append(jpgImage)
print(len(data['crack']))

10000


In [6]:
#Uncrack Data
src = '/content/drive/MyDrive/딥러닝 오소리 /Data Set/10000 DataSet/Negative'
for jpgImage in glob.iglob(os.path.join(src, '*jpg')):
    data['uncrack'].append(jpgImage)
print(len(data['uncrack']))

10000


In [7]:
#훈련에 필요한 파일 생성
os.mkdir('master_data_concrete')
os.mkdir('master_data_concrete/training')
os.mkdir('master_data_concrete/validation')
os.mkdir('master_data_concrete/test')

os.mkdir('master_data_concrete/training/crack')
os.mkdir('master_data_concrete/training/uncrack')
os.mkdir('master_data_concrete/validation/crack')
os.mkdir('master_data_concrete/validation/uncrack')
os.mkdir('master_data_concrete/test/crack')
os.mkdir('master_data_concrete/test/uncrack')

In [8]:
#from sklearn.model_selection import train_test_split
#from sklearn.model_selection import KFold

#kfold = KFold(n_splits=20, shuffle=True)
#rack = data['crack']
#uncrack = data['uncrack']
#X_train, X_valid, y_train, y_valid = train_test_split(crack, uncrack, test_size=0.4, random_state=42)
#X_valid, X_test,y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, random_state=42)

#train_images_list = X_train
#validation_images_list = X_valid

In [ ]:
#60% = training data // 40% = validation data 각각 이미지 파일 복사
#이 과정은 직접 이미지 파일을 사전에 복사 함 으로써 해결가능 그러나 그렇게 되면 능동적인 활용은 안된다.
from sklearn.model_selection import train_test_split

#데이터를 하나로 묶기 ,라벨 vs 데이터
#random 모듈 
crack = data['crack']
uncrack = data['uncrack']
X_train, X_test, y_train, y_test = train_test_split(crack, uncrack, test_size=0.2, random_state=42) #수정 => creak 데이터 셔플  따로 
X_train, X_test, y_train, y_test = train_test_split(crack, uncrack, test_size=0.2, random_state=42) #수정 => creak 라벨 셔플 따로 
X_train, X_test, y_train, y_test = train_test_split(crack, uncrack, test_size=0.2, random_state=42) #수정 => uncreak 데이터 셔플  따로 
X_valid, X_test,y_valid, y_test = train_test_split(X_train, y_train, test_size=0.5, random_state=42) #uncreak 라벨 셔플  따로
#assert my_val is true 구문 추가
#데이터 갯수 확인 코드부분 추가

#만약 처음부터 crack uncrack 데이터를 섞으면 쓸 필요는 없다

In [8]:
#image copy

train_images_list = X_train
validation_images_list = X_valid
test_images_list = X_test

base_dest = 'master_data_concrete'
    #copy training  images of crack type
for image in train_images_list:
        dest = os.path.join(base_dest, 'training')
        shutil.copy(image, dest)

    #copy testing images of crack type
for image in validation_images_list:
        dest = os.path.join(base_dest, 'validation')
        shutil.copy(image, dest)
    
    #copy testing images of crack type
for image in test_images_list:
        dest = os.path.join(base_dest, 'test')
        shutil.copy(image, dest)

In [9]:
#Model Library
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras.callbacks import EarlyStopping, LambdaCallback
from keras.layers import Activation, Dropout,Input, Flatten, Dense
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator

In [10]:
#Convolutional Layer
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(227, 227, 3)))
model.add(BatchNormalization(axis=-1)) #배치 정규화
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [11]:
#Output Layer
model.add(Flatten())  #3D를 1D로 벡터 변환
model.add(Dense(256))
model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1)) 
model.add(BatchNormalization(axis=-1))#this ===== 
model.add(Activation('sigmoid'))  #or model.add(Activation('softmax')) // two output softmax == one output sigmoid

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 225, 225, 32)      896       
_________________________________________________________________
batch_normalization_6 (Batch (None, 225, 225, 32)      128       
_________________________________________________________________
activation_6 (Activation)    (None, 225, 225, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 112, 112, 32)      0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 110, 110, 32)      9248      
_________________________________________________________________
batch_normalization_7 (Batch (None, 110, 110, 32)      128       
_________________________________________________________________
activation_7 (Activation)    (None, 110, 110, 32)     

In [14]:
from keras.preprocessing.image import ImageDataGenerator

batch_size = 30
epochs = 20

#스케일 조정 및 과적합 방지
train_datagen = ImageDataGenerator (
        rescale=1./255, 
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

#validation,test에 사용할 확장구성
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#train, validation, test 모든 이미지 크기 = 227*227
train_generator = train_datagen.flow_from_directory(
        '/content/master_data_concrete/training',  
        target_size=(227, 227), 
        batch_size=batch_size,
          #gray scale
        class_mode='binary')   #binary_crossentropy loss를 사용하기 위한 binary label

#검증 데이터
validation_generator = validation_datagen.flow_from_directory(
        '/content/master_data_concrete/validation',
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='binary')

#테스트 데이터
test_generator = test_datagen.flow_from_directory(
        '/content/master_data_concrete/test',
        target_size=(227, 227),
        batch_size=batch_size,
        class_mode='binary')

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [ ]:
#모델 저장하기 
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
#만약 overfitting, underfitting 을 일으키면 EarlyStopping을 쓰기 위한 코드
#patience가 5면 validation score가 5회 이상 개선이 없을때 훈련을 멈춘다

#from keras.callbacks import EarlyStopping
#es = EarlyStopping(monitor='val_accuracy', patience=5, min_delta=0.005)

In [ ]:
#Model Learning
#콜백함수 적용
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #rmsprop or adam , accuracy or f1-score  (positive와 negative의 데이터 값 차이가 높을 경우)
history = model.fit_generator(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator) #callbacks=[es])

In [ ]:
history = model.fit_generator(
        train_generator,
        epochs=epochs,
        validation_data=validation_generator) 

In [ ]:
#훈련 및 검증 정확도 시각화
#학습 정확성 값과 검증 정확성 값
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'validation'], loc='upper left')
plt.show()

# 학습 손실 값과 검증 손실 값
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'validation'], loc='upper left')
plt.show()

# 테스트 정확성 값과 손실 값
plt.plot(history.history['test_accuracy'])
plt.plot(history.history['test_loss'])
plt.title('Model test')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()
model.save_weights('first_try.h5')

In [ ]:
model.evaluate_generator(validation_generator)